In [1]:
import numpy as np
from numpy import linalg as LA
from numpy.linalg import inv
import math

import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import cm
import matplotlib.patches as mpatches

from sklearn import metrics
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve

# Load data data
ground_truth_poses = np.loadtxt("../data/experiments/pose_scanner_leica_affine_apartment.txt") # id,x,y,z,w,x,y,z
ground_truth_poses = np.array([ground_truth_poses]).T        

corrected_poses = np.loadtxt("../data/experiments/corrected_poses_360fov_apartment_1200_run1.txt")              # idA,idB,x,y,z,w,x,y,z
corrected_poses = np.array([corrected_poses]).T

results = np.loadtxt("../data/experiments/compare_results_360fov_apartment_1200_run1.txt")                     # idA,idB,
results = np.array([results]).T                                                     # fov_overlap, octree_overlap
                                                                                    # alignability, alignment_risk
                                                                                    # degeneracy, ICN
# ground_truth_poses
# print results[5]

In [2]:
# Compute translation error
def translationError(xg, yg, zg, xc, yc, zc):
   
    trans_err = np.zeros((xc.shape[0],1))
    gt_index = 0
    for i in range(0, xc.shape[0]):
        if (gt_index > xg.shape[0]-1):
            gt_index = 0
        
#         print gt_index, i
  
        trans_err[i] =  math.sqrt( pow((xg[gt_index] - xc[i]),2.0) + 
                                   pow((yg[gt_index] - yc[i]),2.0) +
                                   pow((zg[gt_index] - zc[i]),2.0) );
        gt_index = gt_index+1
    return trans_err

transl_err = translationError(ground_truth_poses[1], # [col][row]
                              ground_truth_poses[2],
                              ground_truth_poses[3],
                              corrected_poses[2],
                              corrected_poses[3],
                              corrected_poses[4])

# transl_err

In [3]:
# Compute rotation error
def quaternion_matrix(quaternion):
    quaternion = quaternion / LA.norm(quaternion)
#     print quaternion
    w = quaternion[0]
    x = quaternion[1]
    y = quaternion[2]
    z = quaternion[3]
    
    tmp = np.array([[1 - 2*y*y - 2*z*z,   2*x*y - 2*z*w,       2*x*z + 2*y*w    ],
                    [2*x*y + 2*z*w,       1 - 2*x*x - 2*z*z,   2*y*z - 2*x*w    ],
                    [2*x*z - 2*y*w,       2*y*z + 2*x*w,       1 - 2*x*x - 2*y*y]])
    result = tmp[:, :, 0]
    return result

def rotationError3D(q1g, q2g, q3g, q4g, q1c, q2c, q3c, q4c):
    rot_err = np.zeros((q1c.shape[0],1))
#     print rot_err.shape
    gt_index = 0
    for i in range(0, q1c.shape[0]):
        if (gt_index > q1g.shape[0]-1):
            gt_index = 0
            
#         print gt_index, 1
        rot_g = quaternion_matrix([q1g[gt_index], q2g[gt_index], q3g[gt_index], q4g[gt_index]])
#         print rot_g
#         print q1g[gt_index]
        rot_c = quaternion_matrix([q1c[i], q2c[i], q3c[i], q4c[i]])
        rot_g_inv = inv(rot_g)
        delta_rot = rot_c * rot_g_inv
        trace_rot = np.trace( delta_rot )
#         print trace_rot
  
        rot_err[i] = np.arccos ( (trace_rot-1)/2 ) * 180.0 / math.pi;
        gt_index = gt_index+1
    return rot_err

rot_err = rotationError3D(ground_truth_poses[4], # [col][row]
                          ground_truth_poses[5],
                          ground_truth_poses[6],
                          ground_truth_poses[7],
                          corrected_poses[5],
                          corrected_poses[6],
                          corrected_poses[7],
                          corrected_poses[8])

# print rot_err

In [4]:
# Create Labels given ground truth and errors
transl_thresh = 0.02 # [m]
rot_thresh = 30

def createLabels(transl_err, rot_err):
    
    labels = np.zeros((transl_err.shape[0],1))

    for i in range(0, labels.shape[0]):
        if transl_err[i] > transl_thresh: #and rot_err[i] > rot_thresh:
            labels[i] = 1.0

    return labels

labels = createLabels(transl_err, rot_err)

labels

array([[ 0.],
       [ 0.],
       [ 0.],
       ..., 
       [ 1.],
       [ 0.],
       [ 0.]])

In [5]:
probs_ours = results[5]
probs_degeneracy = results[6]
probs_icn = results[7]

# ROC Curve and cut-off point
fpr_ours, tpr_ours, threshold_ours = metrics.roc_curve(labels, probs_ours, pos_label=1)
roc_ours_auc = metrics.auc(fpr_ours, tpr_ours)

fpr_degeneracy, tpr_degeneracy, threshold_degeneracy = metrics.roc_curve(labels, probs_degeneracy, pos_label=1)
roc_degeneracy_auc = metrics.auc(fpr_degeneracy, tpr_degeneracy)

fpr_icn, tpr_icn, threshold_icn = metrics.roc_curve(labels, probs_icn, pos_label=1)
roc_icn_auc = metrics.auc(fpr_icn, tpr_icn)

# Plot ROC Curve
plt.title('ROC for Failure Prediction using Alignment Risk')
plt.plot(fpr_ours, tpr_ours, 'r', linewidth=2.0, label = 'Ours AUC = %0.2f' % roc_ours_auc)
# plt.plot(fpr_degeneracy, tpr_degeneracy, 'b', linewidth=2.0, label = 'Degeneracy AUC = %0.2f' % roc_degeneracy_auc)
# plt.plot(fpr_icn, tpr_icn, 'g', linewidth=2.0, label = 'ICN AUC = %0.2f' % roc_icn_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

In [6]:
# Precision-Recall Score
average_precision = average_precision_score(labels, probs_ours)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

Average precision-recall score: 0.86


In [7]:
# Plot Precision-Recall Curve
fig = plt.figure()
precision, recall, _ = precision_recall_curve(labels, probs_ours, pos_label=1)

plt.step(recall, precision, color='r',
         where='post', linewidth=2.0, label = 'Ours AUC = %0.2f' % average_precision)
plt.legend(loc = 'lower right')
# plt.fill_between(recall, precision, step='post', alpha=0.2,
#                  color='b')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision-Recall for Failure Prediction using Alignment Risk')

In [8]:
risk_threshold = 0.50

# Fill error matrix
def samplemat(dims, labels, probs_ours):
    matrix = np.ones(dims)
    line_count = 0
    for i in range(min(dims)):
        for j in range(min(dims)):
#             print labels[line_count], probs_ours[line_count]
            if (labels[line_count]==1.0 and probs_ours[line_count]>risk_threshold):
                matrix[i, j] = 1.0 # true positive (dark green)
            elif (labels[line_count]==0.0 and probs_ours[line_count]<=risk_threshold):
                matrix[i, j] = 0.7 # true negative (light green)
            elif (labels[line_count]==0.0 and probs_ours[line_count]>risk_threshold):
                matrix[i, j] = 0.0 # false positive (dark red)
            else:
                matrix[i, j] = 0.1 # false negative (light red)
            line_count = line_count + 1
    return matrix

In [9]:
#####################################
# Plot Failure Prediction Map (ours)
#####################################
fig = plt.figure()
ax = fig.add_subplot(111)
cmap = cm.get_cmap('RdYlGn')

ax.set_xlabel('Cloud A')
ax.set_ylabel('Cloud B')
ax.xaxis.set_label_position('top')
# ax.text(15, 32, 'Failure Predictions using Alignment Risk', color='black', horizontalalignment='center',
#          fontsize=15)
# ax.text(15, 33.5, 'TP: dark green, TN: light green, FP: dark red, FN: light red', color='black', horizontalalignment='center',
#          fontsize=15)
ax.text(23, 47, 'Failure Predictions using Alignment Risk', color='black', horizontalalignment='center',
         fontsize=15)
# ax.text(23, 48.5, 'TP: dark green, TN: light green, FP: dark red, FN: light red', color='black', horizontalalignment='center',
#          fontsize=15)

# ax.text(0, 33, 'True Positives', color=cmap(1.0), horizontalalignment='center',
#          fontsize=11, weight='bold')
# ax.text(4, 33, 'True Negatives', color=cmap(0.7), horizontalalignment='center',
#          fontsize=11, weight='bold')

# Display matrix
mat_size = int(math.sqrt(labels.shape[0]))
# pass octree-overlap column to be taken from results
matrix = samplemat((mat_size, mat_size), labels, probs_ours)
cax = ax.matshow(matrix, cmap=cmap)
# ax.text(23, 1.5, 'TP', color=cmap(1.0), bbox={'facecolor': 'white', 'pad': 10})

# Minor ticks
ax.set_xticks(np.arange(-.5, mat_size-1, 1), minor=True);
ax.set_yticks(np.arange(-.5, mat_size-1, 1), minor=True);

# Gridlines based on minor ticks
# ax.grid(which='minor', color='k')

# print threshold_ours
# plt.show()

In [18]:
# deg_threshold = 0.055 #stairs
deg_threshold = 0.03 #apartment

# Fill error matrix
def samplematDeg(dims, labels, probs_degeneracy, results):
    matrix = np.ones(dims)
    line_count = 0
    for i in range(min(dims)):
        for j in range(min(dims)):
#             print labels[line_count], probs_ours[line_count]
#             print i, j, results[0][line_count], results[1][line_count]
            if (labels[line_count]==1.0 and probs_degeneracy[line_count]<deg_threshold):
                matrix[i, j] = 1.0 # true positive (dark green)
            elif (labels[line_count]==0.0 and probs_degeneracy[line_count]>=deg_threshold):
                matrix[i, j] = 0.7 # true negative (light green)
            elif (labels[line_count]==0.0 and probs_degeneracy[line_count]<deg_threshold):
                matrix[i, j] = 0.0 # false positive (dark red)
            else:
                matrix[i, j] = 0.1 # false negative (light red)
            line_count = line_count + 1
    return matrix

# print probs_degeneracy
##########################################
# Plot Failure Prediction Map (degeneracy)
##########################################
fig = plt.figure()
ax = fig.add_subplot(111)
cmap = cm.get_cmap('RdYlGn')

ax.set_xlabel('Cloud A')
ax.set_ylabel('Cloud B')
ax.xaxis.set_label_position('top')
# ax.text(15, 32, 'Failure Predictions using Degeneracy', color='black', horizontalalignment='center',
#          fontsize=15)
# ax.text(15, 33.5, 'TP: dark green, TN: light green, FP: dark red, FN: light red', color='black', horizontalalignment='center',
#          fontsize=15)
ax.text(23, 47, 'Failure Predictions using Degeneracy', color='black', horizontalalignment='center',
         fontsize=15)
# ax.text(23, 48.5, 'TP: dark green, TN: light green, FP: dark red, FN: light red', color='black', horizontalalignment='center',
#          fontsize=15)

# Display matrix
mat_size = int(math.sqrt(labels.shape[0]))
print mat_size
# pass octree-overlap column to be taken from results
matrix = samplematDeg((mat_size, mat_size), labels, probs_degeneracy, results)
cax = ax.matshow(matrix, cmap=cmap)
# ax.text(23, 1.5, 'TP', color=cmap(1.0), bbox={'facecolor': 'white', 'pad': 10})

# Minor ticks
ax.set_xticks(np.arange(-.5, mat_size-1, 1), minor=True);
ax.set_yticks(np.arange(-.5, mat_size-1, 1), minor=True);

# Gridlines based on minor ticks
# ax.grid(which='minor', color='k')

45


In [ ]:
# icn_threshold = 0.03 #stairs
icn_threshold = 0.05 #apartment

# Fill error matrix
def samplematICN(dims, labels, probs_icn, results):
    matrix = np.ones(dims)
    line_count = 0
    for i in range(min(dims)):
        for j in range(min(dims)):
#             print labels[line_count], probs_ours[line_count]
#             print i, j, results[0][line_count], results[1][line_count]
            if (labels[line_count]==1.0 and probs_icn[line_count]<icn_threshold):
                matrix[i, j] = 1.0 # true positive (dark green)
            elif (labels[line_count]==0.0 and probs_icn[line_count]>=icn_threshold):
                matrix[i, j] = 0.7 # true negative (light green)
            elif (labels[line_count]==0.0 and probs_icn[line_count]<icn_threshold):
                matrix[i, j] = 0.0 # false positive (dark red)
            else:
                matrix[i, j] = 0.1 # false negative (light red)
            line_count = line_count + 1
    return matrix

print probs_icn
##########################################
# Plot Failure Prediction Map (ICN)
##########################################
fig = plt.figure()
ax = fig.add_subplot(111)
cmap = cm.get_cmap('RdYlGn')

ax.set_xlabel('Cloud A')
ax.set_ylabel('Cloud B')
ax.xaxis.set_label_position('top')
# ax.text(15, 32, 'Failure Predictions using ICN', color='black', horizontalalignment='center',
#          fontsize=15)
# ax.text(15, 33.5, 'TP: dark green, TN: light green, FP: dark red, FN: light red', color='black', horizontalalignment='center',
#          fontsize=15)
ax.text(23, 47, 'Failure Predictions using ICN', color='black', horizontalalignment='center',
         fontsize=15)
# ax.text(23, 48.5, 'TP: dark green, TN: light green, FP: dark red, FN: light red', color='black', horizontalalignment='center',
#          fontsize=15)

# Display matrix
mat_size = int(math.sqrt(labels.shape[0]))
print mat_size
# pass octree-overlap column to be taken from results
matrix = samplematICN((mat_size, mat_size), labels, probs_icn, results)
cax = ax.matshow(matrix, cmap=cmap)
# ax.text(23, 1.5, 'TP', color=cmap(1.0), bbox={'facecolor': 'white', 'pad': 10})

# Minor ticks
ax.set_xticks(np.arange(-.5, mat_size-1, 1), minor=True);
ax.set_yticks(np.arange(-.5, mat_size-1, 1), minor=True);

# Gridlines based on minor ticks
# ax.grid(which='minor', color='k')

plt.show()

[[ 0.0677487 ]
 [ 0.100438  ]
 [ 0.074081  ]
 ..., 
 [ 0.00385641]
 [ 0.0507465 ]
 [ 0.026204  ]]
45
